In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [7]:
anon = pd.read_csv('../Data_Anom/Sibyl_483', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)
print(anon.head(50))

    id                date  latitude  longitude  week
0   28 2015-03-27 01:23:00  4.872273  45.781670    13
1   28 2015-03-20 03:59:00  4.656665  46.076388    12
2   50 2015-03-07 23:01:00  4.872410  45.784635    10
3   13 2015-03-22 01:47:00  4.926182  45.779995    12
4   50 2015-05-07 22:35:00  4.872170  45.784588    19
5   51 2015-03-15 00:13:00  4.871645  45.784940    11
6   13 2015-03-13 00:23:00  4.926303  45.779917    11
7   50 2015-03-10 12:17:00  4.872352  45.784553    11
8   49 2015-03-31 12:34:00  4.878358  45.780145    14
9   13 2015-03-06 03:12:00 -0.571607  44.849930    10
10  13 2015-03-22 01:47:00  4.926027  45.780073    12
11  55 2015-03-07 23:01:00  6.404972  46.209265    10
12  27 2015-03-15 00:13:00  4.891630  45.765473    11
13  28 2015-03-24 10:03:00  4.656642  46.076365    13
15  38 2015-03-06 03:12:00  4.875688  45.780507    10
16  27 2015-03-07 23:01:00  4.891767  45.765337    10
17  38 2015-03-13 00:23:00  4.875543  45.780470    11
18  38 2015-03-13 00:23:00  

In [6]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['1' '107' '11' '110' '13' '14' '15' '16' '17' '18' '2' '21' '23' '24'
 '25' '26' '27' '28' '29' '30' '31' '32' '34' '35' '36' '37' '38' '39' '4'
 '41' '42' '43' '44' '48' '49' '5' '50' '51' '52' '53' '54' '55' '58' '59'
 '6' '60' '62' '63' '65' '66' '67' '68' '69' '7' '70' '71' '72' '73' '75'
 '77' '78' '8' '81' '83' '84' '87' '89' '9' '98'] 69


In [9]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [11]:
print(mean_lat_ori)
print(mean_long_ori)
print(mean_date_ori) 
print(count_ligne_ori)
print(mean_hour_ori)

id   week
1    10      4.827873
     11      4.876658
     12      3.315934
     13      4.411797
     14      4.875356
               ...   
107  10      4.870500
     11      4.764978
     12      4.879813
110  10     -0.119641
     11      2.344631
Name: latitude, Length: 449, dtype: float64
id   week
1    10      45.592419
     11      45.781152
     12      44.737925
     13      45.429105
     14      45.774198
               ...    
107  10      45.774327
     11      45.862149
     12      45.786341
110  10      48.404082
     11      48.884047
Name: longitude, Length: 449, dtype: float64
id   week
1    10     2015-03-05 21:39:07.527484928
     11     2015-03-13 14:25:11.006843136
     12     2015-03-19 23:50:54.719796480
     13     2015-03-27 05:09:54.252334592
     14     2015-04-02 16:30:12.192103680
                         ...             
107  10     2015-03-07 04:27:02.706065408
     11     2015-03-13 22:06:22.832617984
     12     2015-03-17 11:12:29.716608000
110  10 

In [12]:
print(mean_lat_anon)
print(mean_long_anon)
print(mean_date_anon) 
print(count_ligne_anon)
print(mean_hour_anon)

id  week
1   10      4.823233
    11      4.876658
    12      4.857129
    13      4.863563
    14      4.875238
              ...   
9   15      4.840804
    16      4.840811
    19      4.840891
98  10      4.868932
    11      4.753122
Name: latitude, Length: 441, dtype: float64
id  week
1   10      45.599683
    11      45.781151
    12      45.761995
    13      45.757418
    14      45.774130
              ...    
9   15      45.770785
    16      45.770833
    19      45.770823
98  10      45.775878
    11      45.868809
Name: longitude, Length: 441, dtype: float64
id  week
1   10     2015-03-05 23:13:48.976227840
    11     2015-03-13 12:04:36.753849600
    12     2015-03-18 00:50:31.772650496
    13     2015-03-27 20:28:32.608391680
    14     2015-04-02 20:11:48.268519936
                        ...             
9   15     2015-04-09 12:48:14.364190464
    16     2015-04-16 22:58:00.000000000
    19     2015-05-05 13:09:00.000000000
98  10     2015-03-06 21:59:10.564334080
 

In [31]:
import numpy as np

for week, group in df.groupby('week'):
    print(f'Week {week}:')
    unique_ids = np.sort(group['id'].unique()) 
    dict_id[week] = list(unique_ids)  
    print("Number of unique IDs real:", len(unique_ids))
    print(unique_ids)
    print()

Week 10:
Number of unique IDs real: 64
[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  54  55  58  59  60  62  63  66  67  68  69  71  72
  73  75  77  78  83  87  89  98 107 110]

Week 11:
Number of unique IDs real: 59
[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  21  23  24  25
  26  27  28  29  30  31  34  36  37  38  41  42  43  44  48  49  50  51
  52  53  54  55  58  59  60  62  63  67  68  69  72  73  75  77  78  83
  87  89  98 107 110]

Week 12:
Number of unique IDs real: 51
[  1   2   4   6   7   8   9  13  15  16  17  18  24  26  27  28  30  31
  32  34  38  41  42  43  44  49  50  51  52  53  55  59  60  62  63  65
  66  67  68  69  70  71  72  73  75  77  78  81  84  87 107]

Week 13:
Number of unique IDs real: 48
[ 1  2  4  6  7  8  9 13 17 18 24 26 27 28 29 30 31 32 35 41 42 43 44 49
 50 51 52 53 54 59 60 62 63 65 66 67 68 69 70 71 72 73 75 

In [92]:
dict_id_ori = {
    str("2015-10"): [
            1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 21, 23, 24,
            25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44,
            48, 49, 50, 51, 52, 54, 55, 58, 59, 60, 62, 63, 66, 67, 68, 69, 71, 72,
            73, 75, 77, 78, 83, 87, 89, 98, 107, 110
        ],
    str("2015-11"): [
            1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 21, 23, 24, 25,
            26, 27, 28, 29, 30, 31, 34, 36, 37, 38, 41, 42, 43, 44, 48, 49, 50, 51,
            52, 53, 54, 55, 58, 59, 60, 62, 63, 67, 68, 69, 72, 73, 75, 77, 78, 83,
            87, 89, 98, 107, 110
        ],
    str("2015-12"): [
            1, 2, 4, 6, 7, 8, 9, 13, 15, 16, 17, 18, 24, 26, 27, 28, 30, 31,
            32, 34, 38, 41, 42, 43, 44, 49, 50, 51, 52, 53, 55, 59, 60, 62, 63, 65,
            66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 78, 81, 84, 87, 107
        ],
    str("2015-13"): [
            1, 2, 4, 6, 7, 8, 9, 13, 17, 18, 24, 26, 27, 28, 29, 30, 31, 32, 35, 41,
            42, 43, 44, 49, 50, 51, 52, 53, 54, 59, 60, 62, 63, 65, 66, 67, 68, 69, 70, 71,
            72, 73, 75, 77, 78, 81, 83, 84
        ],
    str("2015-14"): [
            1, 2, 4, 6, 7, 8, 9, 13, 17, 24, 27, 28, 29, 30, 31, 32, 35, 41, 42, 49,
            50, 51, 52, 53, 54, 59, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 78,
            81, 83, 84
        ],
    str("2015-15"): [
            1, 2, 4, 6, 7, 8, 9, 13, 18, 24, 26, 27, 30, 31, 32, 35, 39, 41, 42, 44,
            49, 50, 51, 59, 62, 63, 65, 66, 67, 68, 69, 71, 72, 73, 75, 81, 83
        ],
    str("2015-16"): [
            1, 2, 4, 6, 7, 8, 9, 13, 18, 24, 26, 27, 30, 31, 32, 35, 36, 39, 41, 42,
            48, 49, 50, 51, 59, 62, 63, 66, 67, 68, 69, 71, 72, 73, 75, 78, 83, 87
        ],
    str("2015-17"): [
            1, 2, 4, 7, 8, 13, 17, 18, 24, 26, 27, 30, 32, 41, 42, 50, 51, 59, 63, 66,
            67, 68, 69, 71, 72, 73, 75, 77, 78, 83, 87
        ],
    str("2015-18"): [
            1, 2, 4, 17, 18, 24, 26, 30, 32, 36, 39, 42, 50, 51, 59, 62, 63, 66, 67, 68,
            69, 71, 72, 73, 75, 77, 78, 83, 87, 89
        ],
    str("2015-19"): [
            1, 2, 4, 9, 13, 17, 18, 24, 26, 30, 32, 39, 42, 43, 50, 51, 58, 59, 60, 62, 
            63, 66, 67, 68, 69, 72, 73, 75, 83, 87
        ],
    str("2015-20"): [
            1, 2, 4, 13, 18, 24, 42, 50, 51, 59, 62, 66, 67, 68, 69, 72, 73, 75, 77
        ]
}

print(dict_id_ori)

{'2015-10': [1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 48, 49, 50, 51, 52, 54, 55, 58, 59, 60, 62, 63, 66, 67, 68, 69, 71, 72, 73, 75, 77, 78, 83, 87, 89, 98, 107, 110], '2015-11': [1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 36, 37, 38, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 58, 59, 60, 62, 63, 67, 68, 69, 72, 73, 75, 77, 78, 83, 87, 89, 98, 107, 110], '2015-12': [1, 2, 4, 6, 7, 8, 9, 13, 15, 16, 17, 18, 24, 26, 27, 28, 30, 31, 32, 34, 38, 41, 42, 43, 44, 49, 50, 51, 52, 53, 55, 59, 60, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 78, 81, 84, 87, 107], '2015-13': [1, 2, 4, 6, 7, 8, 9, 13, 17, 18, 24, 26, 27, 28, 29, 30, 31, 32, 35, 41, 42, 43, 44, 49, 50, 51, 52, 53, 54, 59, 60, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 78, 81, 83, 84], '2015-14': [1, 2, 4, 6, 7, 8, 9, 13, 17, 24, 27, 28, 29, 30, 31, 32, 35, 41, 42, 49,

In [93]:
for week, group in anon.groupby('week'):
    print(f'Week {week}:')
    unique_ids_anon = np.sort(group['id'].unique()) 
    dict_id_anon[week] = list(unique_ids_anon)
    print("Number of unique IDs anon:", len(unique_ids_anon))
    print(unique_ids_anon)
    print()

Week 10:
Number of unique IDs anon: 63
['1' '107' '11' '110' '13' '14' '15' '16' '17' '18' '2' '21' '23' '24'
 '25' '26' '27' '28' '29' '30' '31' '32' '34' '35' '36' '37' '38' '39' '4'
 '41' '42' '43' '44' '48' '49' '5' '50' '51' '52' '54' '55' '59' '6' '60'
 '62' '63' '66' '67' '68' '69' '7' '71' '72' '73' '75' '77' '78' '8' '83'
 '87' '89' '9' '98']

Week 11:
Number of unique IDs anon: 58
['1' '107' '11' '110' '13' '14' '15' '16' '17' '2' '21' '23' '24' '25'
 '26' '27' '28' '29' '30' '31' '34' '36' '37' '38' '4' '41' '42' '43' '44'
 '48' '49' '5' '50' '51' '52' '53' '54' '55' '59' '6' '60' '62' '63' '67'
 '68' '69' '7' '72' '73' '75' '77' '78' '8' '83' '87' '89' '9' '98']

Week 12:
Number of unique IDs anon: 51
['1' '107' '13' '15' '16' '17' '18' '2' '24' '26' '27' '28' '30' '31' '32'
 '34' '38' '4' '41' '42' '43' '44' '49' '50' '51' '52' '53' '55' '59' '6'
 '60' '62' '63' '65' '66' '67' '68' '69' '7' '70' '71' '72' '73' '75' '77'
 '78' '8' '81' '84' '87' '9']

Week 13:
Number of uni

In [120]:
dict_id_anon = {
    str("2015-10"): [
            '1', '107', '11', '110', '13', '14', '15', '16', '17', '18', '2', '21', '23', '24',
            '25', '26', '27', '28', '29', '30', '31', '32', '34', '35', '36', '37', '38', '39', '4',
            '41', '42', '43', '44', '48', '49', '5', '50', '51', '52', '54', '55', '59', '6', '60',
            '62', '63', '66', '67', '68', '69', '7', '71', '72', '73', '75', '77', '78', '8', '83',
            '87', '89', '9', '98'
        ],
    str("2015-11"): [
            '1', '107', '11', '110', '13', '14', '15', '16', '17', '2', '21', '23', '24', '25',
            '26', '27', '28', '29', '30', '31', '34', '36', '37', '38', '4', '41', '42', '43', '44',
            '48', '49', '5', '50', '51', '52', '53', '54', '55', '59', '6', '60', '62', '63', '67',
            '68', '69', '7', '72', '73', '75', '77', '78', '8', '83', '87', '89', '9', '98'
        ],
    str("2015-12"): [
            '1', '107', '13', '15', '16', '17', '18', '2', '24', '26', '27', '28', '30', '31', '32',
            '34', '38', '4', '41', '42', '43', '44', '49', '50', '51', '52', '53', '55', '59', '6',
            '60', '62', '63', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '75', '77',
            '78', '8', '81', '84', '87', '9'
        ],
    str("2015-13"): [
            '1', '13', '17', '18', '2', '24', '26', '27', '28', '29', '30', '32', '35', '4', '41',
            '42', '43', '44', '49', '50', '51', '52', '53', '54', '59', '6', '60', '62', '63', '65',
            '66', '67', '68', '69', '7', '70', '71', '72', '73', '75', '77', '78', '8', '81', '83',
            '84', '9'
        ],
    str("2015-14"): [
            '1', '13', '17', '2', '24', '27', '28', '29', '30', '31', '32', '35', '4', '41', '42',
            '49', '50', '51', '52', '53', '54', '59', '6', '62', '63', '65', '66', '67', '68', '69',
            '7', '70', '71', '72', '73', '75', '78', '8', '81', '83', '84', '9'
        ],
    str("2015-15"): [
            '1', '13', '18', '2', '24', '26', '27', '30', '31', '32', '35', '39', '4', '41', '42',
            '44', '49', '50', '51', '59', '6', '62', '63', '65', '66', '67', '68', '69', '7', '71',
            '72', '73', '75', '8', '81', '83', '9'
        ],
    str("2015-16"): [
            '1', '13', '18', '2', '24', '26', '27', '30', '31', '32', '35', '36', '39', '4', '41',
            '42', '48', '49', '50', '51', '59', '6', '63', '66', '67', '68', '69', '7', '71', '72',
            '73', '75', '78', '8', '83', '87', '9'
        ],
    str("2015-17"): [
            '1', '13', '17', '18', '2', '24', '26', '27', '30', '32', '4', '41', '42', '50', '51',
            '59', '63', '66', '67', '68', '69', '7', '71', '72', '73', '75', '77', '78', '8', '83',
            '87'
        ],
    str("2015-18"): [
            '1', '17', '2', '24', '26', '30', '32', '36', '39', '4', '42', '50', '51', '59', '62',
            '66', '67', '68', '69', '71', '72', '73', '75', '77', '78', '83', '89'
        ],
    str("2015-19"): [
            '1', '13', '17', '18', '2', '24', '26', '30', '32', '39', '4', '42', '43', '50', '51',
            '58', '59', '60', '62', '63', '66', '67', '68', '69', '72', '73', '75', '83', '87', '9'
        ],
    str("2015-20"): [
            '13', '18', '2', '24', '4', '42', '50', '51', '59', '62', '66', '67', '68', '69', '72',
            '73', '75', '77'
        ]
}

print(dict_id_anon)

{'2015-10': ['1', '107', '11', '110', '13', '14', '15', '16', '17', '18', '2', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '34', '35', '36', '37', '38', '39', '4', '41', '42', '43', '44', '48', '49', '5', '50', '51', '52', '54', '55', '59', '6', '60', '62', '63', '66', '67', '68', '69', '7', '71', '72', '73', '75', '77', '78', '8', '83', '87', '89', '9', '98'], '2015-11': ['1', '107', '11', '110', '13', '14', '15', '16', '17', '2', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '34', '36', '37', '38', '4', '41', '42', '43', '44', '48', '49', '5', '50', '51', '52', '53', '54', '55', '59', '6', '60', '62', '63', '67', '68', '69', '7', '72', '73', '75', '77', '78', '8', '83', '87', '89', '9', '98'], '2015-12': ['1', '107', '13', '15', '16', '17', '18', '2', '24', '26', '27', '28', '30', '31', '32', '34', '38', '4', '41', '42', '43', '44', '49', '50', '51', '52', '53', '55', '59', '6', '60', '62', '63', '65', '66', '67', '68', '69', '7', '70', '71', '72',

In [121]:
for key, value in dict_id_anon.items():
    dict_id_anon[key] = [int(x) for x in ' '.join(map(str, np.sort([int(x) for x in value]))).split()]

In [122]:
print(dict_id_anon)

{'2015-10': [1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 48, 49, 50, 51, 52, 54, 55, 59, 60, 62, 63, 66, 67, 68, 69, 71, 72, 73, 75, 77, 78, 83, 87, 89, 98, 107, 110], '2015-11': [1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 36, 37, 38, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 59, 60, 62, 63, 67, 68, 69, 72, 73, 75, 77, 78, 83, 87, 89, 98, 107, 110], '2015-12': [1, 2, 4, 6, 7, 8, 9, 13, 15, 16, 17, 18, 24, 26, 27, 28, 30, 31, 32, 34, 38, 41, 42, 43, 44, 49, 50, 51, 52, 53, 55, 59, 60, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 78, 81, 84, 87, 107], '2015-13': [1, 2, 4, 6, 7, 8, 9, 13, 17, 18, 24, 26, 27, 28, 29, 30, 32, 35, 41, 42, 43, 44, 49, 50, 51, 52, 53, 54, 59, 60, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 78, 81, 83, 84], '2015-14': [1, 2, 4, 6, 7, 8, 9, 13, 17, 24, 27, 28, 29, 30, 31, 32, 35, 41, 42, 49, 50, 51, 52,

In [97]:
print(type(dict_id_anon[key]))

<class 'list'>


In [98]:
weeks = []

for key in dict_id_ori:
    weeks.append(key)

result = {}

for week in weeks:
    pairs = []

    for e in dict_id_ori[week]:
        
        if e in dict_id_anon[week]:
            pairs.append((e, e))
        else:
            pairs.append((e, 'DEL'))

    result[week] = pairs
    print(week, pairs)
    print(len(pairs))

2015-10 [(1, 1), (2, 2), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (11, 11), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (21, 21), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (41, 41), (42, 42), (43, 43), (44, 44), (48, 48), (49, 49), (50, 50), (51, 51), (52, 52), (54, 54), (55, 55), (58, 'DEL'), (59, 59), (60, 60), (62, 62), (63, 63), (66, 66), (67, 67), (68, 68), (69, 69), (71, 71), (72, 72), (73, 73), (75, 75), (77, 77), (78, 78), (83, 83), (87, 87), (89, 89), (98, 98), (107, 107), (110, 110)]
64
2015-11 [(1, 1), (2, 2), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (11, 11), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (21, 21), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (34, 34), (36, 36), (37, 37), (38, 38), (41, 41), (42, 42), (43, 43), (44, 44), (48, 48), (49, 49), (50, 50), (51, 51), (52,

In [99]:
print(result)

{'2015-10': [(1, 1), (2, 2), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (11, 11), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (21, 21), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (41, 41), (42, 42), (43, 43), (44, 44), (48, 48), (49, 49), (50, 50), (51, 51), (52, 52), (54, 54), (55, 55), (58, 'DEL'), (59, 59), (60, 60), (62, 62), (63, 63), (66, 66), (67, 67), (68, 68), (69, 69), (71, 71), (72, 72), (73, 73), (75, 75), (77, 77), (78, 78), (83, 83), (87, 87), (89, 89), (98, 98), (107, 107), (110, 110)], '2015-11': [(1, 1), (2, 2), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (11, 11), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (21, 21), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (34, 34), (36, 36), (37, 37), (38, 38), (41, 41), (42, 42), (43, 43), (44, 44), (48, 48), (49, 49), (50, 50), (51, 51),

In [133]:
attack_Sibyl_483_json = {}
for unique_id in list_id_ori:
    guess_id_in_all_weeks = {}
    for week in result:
        # get the pair list for one week
        pairs = result[week]
        # find the unique id in the pair list in this week
        for pair in pairs:
            if pair[0] == unique_id:
                guess_id_in_all_weeks[week] = [str(pair[1])]

        for week in result:
            if week not in guess_id_in_all_weeks:
                guess_id_in_all_weeks[week] = ["DEL"]
    
    new_guess_id_in_all_weeks = {str(key): value for key, value in guess_id_in_all_weeks.items()}

    
    attack_Sibyl_483_json[str(unique_id)] = new_guess_id_in_all_weeks

print(attack_Sibyl_483_json)

{'1': {'2015-10': ['1'], '2015-11': ['1'], '2015-12': ['1'], '2015-13': ['1'], '2015-14': ['1'], '2015-15': ['1'], '2015-16': ['1'], '2015-17': ['1'], '2015-18': ['1'], '2015-19': ['1'], '2015-20': ['DEL']}, '2': {'2015-10': ['2'], '2015-11': ['2'], '2015-12': ['2'], '2015-13': ['2'], '2015-14': ['2'], '2015-15': ['2'], '2015-16': ['2'], '2015-17': ['2'], '2015-18': ['2'], '2015-19': ['2'], '2015-20': ['2']}, '107': {'2015-10': ['107'], '2015-11': ['107'], '2015-12': ['107'], '2015-13': ['DEL'], '2015-14': ['DEL'], '2015-15': ['DEL'], '2015-16': ['DEL'], '2015-17': ['DEL'], '2015-18': ['DEL'], '2015-19': ['DEL'], '2015-20': ['DEL']}, '98': {'2015-10': ['98'], '2015-11': ['98'], '2015-12': ['DEL'], '2015-13': ['DEL'], '2015-14': ['DEL'], '2015-15': ['DEL'], '2015-16': ['DEL'], '2015-17': ['DEL'], '2015-18': ['DEL'], '2015-19': ['DEL'], '2015-20': ['DEL']}, '21': {'2015-10': ['21'], '2015-11': ['21'], '2015-12': ['DEL'], '2015-13': ['DEL'], '2015-14': ['DEL'], '2015-15': ['DEL'], '2015-1

In [134]:
jsonData = json.dumps(attack_Sibyl_483_json, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "1"
        ],
        "2015-11": [
            "1"
        ],
        "2015-12": [
            "1"
        ],
        "2015-13": [
            "1"
        ],
        "2015-14": [
            "1"
        ],
        "2015-15": [
            "1"
        ],
        "2015-16": [
            "1"
        ],
        "2015-17": [
            "1"
        ],
        "2015-18": [
            "1"
        ],
        "2015-19": [
            "1"
        ],
        "2015-20": [
            "DEL"
        ]
    },
    "2": {
        "2015-10": [
            "2"
        ],
        "2015-11": [
            "2"
        ],
        "2015-12": [
            "2"
        ],
        "2015-13": [
            "2"
        ],
        "2015-14": [
            "2"
        ],
        "2015-15": [
            "2"
        ],
        "2015-16": [
            "2"
        ],
        "2015-17": [
            "2"
        ],
        "2015-18": [
            "2"
        ],
        